In [1]:
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10, sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [3]:
X, y = random_sum_pairs(100, 4, 20)
print(X)

[[12, 10, 10, 12], [15, 15, 10, 2], [20, 19, 16, 7], [9, 2, 19, 12], [7, 2, 9, 6], [2, 11, 19, 10], [13, 7, 5, 13], [8, 3, 3, 2], [1, 4, 12, 8], [15, 8, 16, 6], [15, 20, 1, 10], [14, 14, 6, 7], [20, 20, 19, 16], [11, 15, 14, 9], [4, 4, 8, 17], [11, 3, 18, 6], [13, 11, 1, 10], [19, 10, 3, 2], [3, 11, 18, 11], [12, 17, 15, 5], [10, 8, 10, 5], [9, 16, 1, 15], [10, 11, 3, 15], [16, 16, 5, 15], [9, 11, 11, 18], [19, 5, 16, 20], [13, 6, 14, 4], [2, 13, 17, 7], [11, 16, 1, 15], [3, 14, 6, 8], [7, 2, 16, 19], [4, 3, 5, 16], [10, 20, 19, 10], [13, 6, 15, 14], [2, 17, 3, 12], [11, 12, 9, 4], [7, 6, 2, 20], [19, 10, 19, 12], [5, 15, 1, 15], [16, 15, 9, 9], [6, 9, 5, 10], [13, 2, 5, 9], [10, 13, 9, 2], [14, 14, 15, 20], [10, 14, 19, 11], [5, 1, 5, 3], [2, 10, 20, 18], [16, 2, 12, 9], [7, 12, 2, 6], [16, 5, 2, 8], [14, 20, 4, 3], [9, 15, 16, 1], [11, 8, 9, 20], [6, 11, 14, 4], [18, 6, 8, 1], [16, 7, 16, 10], [15, 16, 3, 12], [18, 2, 13, 4], [5, 11, 20, 14], [4, 14, 8, 3], [19, 7, 14, 19], [1, 8, 17

In [4]:
print(y)

[44, 42, 62, 42, 24, 42, 38, 16, 25, 45, 46, 41, 75, 49, 33, 38, 35, 34, 43, 49, 33, 41, 39, 52, 49, 60, 37, 39, 43, 31, 44, 28, 59, 48, 34, 36, 35, 60, 36, 49, 30, 29, 34, 63, 54, 14, 50, 39, 27, 31, 41, 41, 48, 35, 33, 49, 46, 37, 50, 29, 59, 32, 40, 20, 45, 47, 56, 20, 37, 47, 44, 18, 60, 42, 50, 54, 63, 44, 35, 54, 42, 23, 31, 55, 37, 52, 26, 48, 55, 43, 48, 44, 61, 19, 36, 47, 37, 48, 54, 47]


In [5]:
ceil(log10(20+1))

2

In [6]:
def pairs_to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = []
    for p in X:
        strp = '+'.join([str(n) for n in p])
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [7]:
pairs_to_string(X, y, 4, 20)

(['12+10+10+12',
  ' 15+15+10+2',
  ' 20+19+16+7',
  '  9+2+19+12',
  '    7+2+9+6',
  ' 2+11+19+10',
  '  13+7+5+13',
  '    8+3+3+2',
  '   1+4+12+8',
  '  15+8+16+6',
  ' 15+20+1+10',
  '  14+14+6+7',
  '20+20+19+16',
  ' 11+15+14+9',
  '   4+4+8+17',
  '  11+3+18+6',
  ' 13+11+1+10',
  '  19+10+3+2',
  ' 3+11+18+11',
  ' 12+17+15+5',
  '  10+8+10+5',
  '  9+16+1+15',
  ' 10+11+3+15',
  ' 16+16+5+15',
  ' 9+11+11+18',
  ' 19+5+16+20',
  '  13+6+14+4',
  '  2+13+17+7',
  ' 11+16+1+15',
  '   3+14+6+8',
  '  7+2+16+19',
  '   4+3+5+16',
  '10+20+19+10',
  ' 13+6+15+14',
  '  2+17+3+12',
  '  11+12+9+4',
  '   7+6+2+20',
  '19+10+19+12',
  '  5+15+1+15',
  '  16+15+9+9',
  '   6+9+5+10',
  '   13+2+5+9',
  '  10+13+9+2',
  '14+14+15+20',
  '10+14+19+11',
  '    5+1+5+3',
  ' 2+10+20+18',
  '  16+2+12+9',
  '   7+12+2+6',
  '   16+5+2+8',
  '  14+20+4+3',
  '  9+15+16+1',
  '  11+8+9+20',
  '  6+11+14+4',
  '   18+6+8+1',
  ' 16+7+16+10',
  ' 15+16+3+12',
  '  18+2+13+4',
  ' 5+11+20+14

In [8]:
Xstr, ystr = pairs_to_string(X, y, 4, 20)

In [9]:
def integer_encode(X, y, alphabets):
    char_to_int = dict((c, i) for i, c in enumerate(alphabets))
    print(char_to_int)
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    yenc = []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [10]:
integer_encode(Xstr, ystr, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+'])

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


([[0, 1, 11, 0, 9, 11, 0, 9, 11, 0, 1],
  [10, 0, 4, 11, 0, 4, 11, 0, 9, 11, 1],
  [10, 1, 9, 11, 0, 8, 11, 0, 5, 11, 6],
  [10, 10, 8, 11, 1, 11, 0, 8, 11, 0, 1],
  [10, 10, 10, 10, 6, 11, 1, 11, 8, 11, 5],
  [10, 1, 11, 0, 0, 11, 0, 8, 11, 0, 9],
  [10, 10, 0, 2, 11, 6, 11, 4, 11, 0, 2],
  [10, 10, 10, 10, 7, 11, 2, 11, 2, 11, 1],
  [10, 10, 10, 0, 11, 3, 11, 0, 1, 11, 7],
  [10, 10, 0, 4, 11, 7, 11, 0, 5, 11, 5],
  [10, 0, 4, 11, 1, 9, 11, 0, 11, 0, 9],
  [10, 10, 0, 3, 11, 0, 3, 11, 5, 11, 6],
  [1, 9, 11, 1, 9, 11, 0, 8, 11, 0, 5],
  [10, 0, 0, 11, 0, 4, 11, 0, 3, 11, 8],
  [10, 10, 10, 3, 11, 3, 11, 7, 11, 0, 6],
  [10, 10, 0, 0, 11, 2, 11, 0, 7, 11, 5],
  [10, 0, 2, 11, 0, 0, 11, 0, 11, 0, 9],
  [10, 10, 0, 8, 11, 0, 9, 11, 2, 11, 1],
  [10, 2, 11, 0, 0, 11, 0, 7, 11, 0, 0],
  [10, 0, 1, 11, 0, 6, 11, 0, 4, 11, 4],
  [10, 10, 0, 9, 11, 7, 11, 0, 9, 11, 4],
  [10, 10, 8, 11, 0, 5, 11, 0, 11, 0, 4],
  [10, 0, 9, 11, 0, 0, 11, 2, 11, 0, 4],
  [10, 0, 5, 11, 0, 5, 11, 4, 11, 0, 4],


In [11]:
def one_hot_encode(X, y, max_int):
    Xenc = []
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = []
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [12]:
def generate_data(n_samples, n_numbers, largest, alphabets):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = pairs_to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabets)
    X, y = one_hot_encode(X, y, len(alphabets))
    X, y = array(X), array(y)
    return X, y

In [13]:
def integer_decode(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets))
    strings = []
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

In [14]:
seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+']
n_chars = len(alphabets)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

In [15]:
n_chars

12

In [16]:
n_in_seq_length

8

In [17]:
n_out_seq_length

2

In [18]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


In [19]:
X.shape

(1000, 8, 12)

In [20]:
X[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 64)             49408     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            780       
 ibuted)                                                         
                                                                 
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________


In [25]:
for i in range(1000000):
    X, y = generate_data(n_samples, n_numbers, largest, alphabets)
    print('Epoch: ', i)
    model.fit(X, y, epochs=1000, batch_size=50)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  0
Epoch 1/5
20/20 [==============================] - 0s 7ms/step - loss: 1.0271 - accuracy: 0.6485
Epoch 2/5
20/20 [==============================] - 0s 7ms/step - loss: 0.9984 - accuracy: 0.6695
Epoch 3/5
20/20 [==============================] - 0s 7ms/step - loss: 1.0032 - accuracy: 0.6435
Epoch 4/5
20/20 [==============================] - 0s 7ms/step - loss: 0.9766 - accuracy: 0.6605
Epoch 5/5
20/20 [==============================] - 0s 7ms/step - loss: 0.9521 - accuracy: 0.6830
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  1
Epoch 1/5
20/20 [==============================] - 0s 7ms/step - loss: 0.9711 - accuracy: 0.6680
Epoch 2/5
20/20 [==============================] - 0s 6ms/step - loss: 0.9447 - accuracy: 0.6780
Epoch 3/5
20/20 [==============================] - 0s 7ms/step - loss: 0.9235 - accuracy: 0.7055
Epoch 

In [26]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)
result = model.predict(X, batch_size=50)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
20/20 [==============================] - 1s 3ms/step


In [27]:
result[0]

array([[5.74203796e-06, 9.99915302e-01, 7.88139660e-05, 1.34872607e-07,
        5.42693057e-10, 5.15063200e-11, 4.42750030e-13, 4.17527558e-13,
        4.14035438e-11, 8.24200619e-09, 6.00222788e-11, 6.94863125e-11],
       [2.47242369e-05, 8.09953199e-05, 3.70647013e-03, 9.24346596e-03,
        9.18966979e-02, 7.82961071e-01, 1.04644045e-01, 6.51125517e-03,
        8.37269647e-04, 9.37927398e-05, 9.17760268e-09, 1.35241010e-07]],
      dtype=float32)

In [28]:
integer_decode(result[0], alphabets)

'26'

In [29]:
expected = [integer_decode(y1, alphabets) for y1 in y]
predicted = [integer_decode(y1, alphabets) for y1 in result]

In [30]:
list(zip(expected, predicted))

[('26', '26'),
 ('50', '50'),
 ('34', '34'),
 ('41', '41'),
 ('32', '32'),
 ('33', '33'),
 ('24', '23'),
 ('26', '26'),
 ('34', '34'),
 ('38', '38'),
 ('20', '20'),
 ('38', '38'),
 ('39', '39'),
 ('42', '42'),
 ('26', '26'),
 ('19', '19'),
 ('36', '36'),
 ('43', '43'),
 ('29', '29'),
 ('20', '20'),
 ('28', '28'),
 ('26', '26'),
 ('51', '51'),
 ('35', '35'),
 ('50', '50'),
 ('20', '20'),
 ('29', '29'),
 ('15', '15'),
 ('39', '39'),
 ('44', '44'),
 ('30', '30'),
 ('36', '36'),
 ('27', '27'),
 ('23', '23'),
 ('21', '21'),
 ('46', '46'),
 ('39', '39'),
 ('54', '53'),
 ('26', '26'),
 ('21', '21'),
 ('18', '18'),
 ('40', '40'),
 ('35', '35'),
 ('47', '47'),
 ('22', '22'),
 ('14', '14'),
 ('31', '31'),
 ('37', '37'),
 ('33', '33'),
 ('34', '34'),
 ('47', '47'),
 ('31', '31'),
 ('22', '22'),
 ('34', '34'),
 ('11', '11'),
 ('22', '22'),
 ('43', '43'),
 ('41', '41'),
 ('30', '30'),
 ('37', '37'),
 ('38', '38'),
 ('26', '26'),
 ('29', '29'),
 ('15', '15'),
 ('29', '29'),
 ('58', '55'),
 ('21', '2

In [31]:
import numpy as np
np.abs(np.array(expected, dtype='int') - np.array(predicted, dtype='int')).sum() / len(expected)

0.071

In [32]:
# MasoudKaviani.ir